In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm
from itertools import chain
from itertools import permutations

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

reverse_mapping = defaultdict(list)
for k,v in repeats.items():
    reverse_mapping[v['map']].append(k)
for k in reverse_mapping:
    reverse_mapping[k].append(k)

mappings = list(chain(*[permutations(ls,2) for ls in reverse_mapping.values()]))

def cost(i,j):
    if (i,j) in mappings:
        return 1e+8
    i,j = get(i), get(j)
    return distmat[i,j]

def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
clusters = json.load(open('./clusters.json'))
list(map(lambda x: len(x), clusters))

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [3]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
        
    for pair in mappings:
        if pair[0] in cluster and pair[1] in cluster:
            pair = Neighbours.index(pair[0])+1, Neighbours.index(pair[1])+1
            #print(Neighbours, pair)
            for length in range(1,6):
                all_groups = permutations(N,length)
                for group in all_groups:
                    if pair[0] in group or pair[1] in group:
                        continue
                    group = [pair[0]] + list(group) + [pair[1]]
                    edges = [(group[i], group[i+1]) for i in range(len(group) - 1)]
                    expr = 0
                    for edge in edges:
                        expr += x[edge]
                    m.addConstr(expr <= length)
    m.Params.timelimit=600
    primary_models.append(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600


In [4]:
for model in tqdm(primary_models):
    model.optimize();

  0%|                                                    | 0/18 [00:00<?, ?it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 122 rows, 133 columns and 492 nonzeros
Model fingerprint: 0x27318550
Variable types: 11 continuous, 122 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 439110.00000
Presolve removed 14 rows and 51 columns
Presolve time: 0.00s
Presolved: 108 rows, 82 columns, 396 nonzeros
Variable types: 10 continuous, 72 integer (72 binary)
Found heuristic solution: objective 204900.00000

Root relaxation: objective 5.767422e+04, 38 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 57674.2222    0   14 204900.000 57674.22

  6%|██▍                                         | 1/18 [00:08<02:20,  8.28s/it]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17722 rows, 133 columns and 100988 nonzeros
Model fingerprint: 0xacd5c657
Variable types: 11 continuous, 122 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [3e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 1285147.0000
Presolve removed 12 rows and 35 columns
Presolve time: 0.23s
Presolved: 17710 rows, 98 columns, 166236 nonzeros
Variable types: 10 continuous, 88 integer (88 binary)
Found heuristic solution: objective 618040.00000

Root relaxation: objective 1.447887e+05, 49 iterations, 0.08 seconds (0.11 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 144788.667    0   21 618040.00

 101436 33867 301023.402   45   14 460257.913 240846.000  47.7%   9.5  421s
 103071 34543 333797.600   61   14 460257.913 241277.200  47.6%   9.5  425s
 105671 35429 437701.500   60    5 460257.913 241813.875  47.5%   9.5  430s
 108078 36476 266092.319   53   20 460257.913 242302.167  47.4%   9.5  435s
 110475 37503 infeasible   46      460257.913 242874.852  47.2%   9.5  440s
 112981 38330 280167.587   48   16 460257.913 243419.889  47.1%   9.5  445s
 115274 39192 337968.378   53   12 460257.913 244048.171  47.0%   9.5  450s
 117923 40113 279563.794   50   16 460257.913 244691.444  46.8%   9.5  455s
 120449 40985 265754.333   53   14 460257.913 245273.690  46.7%   9.5  460s
 122882 41701 437453.248   53    7 460257.913 245944.991  46.6%   9.5  465s
 126075 42794 297137.333   63   19 460257.913 246843.127  46.4%   9.5  471s
 128533 43643 381397.019   51    9 460257.913 247421.215  46.2%   9.5  476s
 131079 44576 305751.194   63   16 460257.913 248148.140  46.1%   9.5  481s
 133450 4545

...

 89%|███████████████████████████████████▌    | 16/18 [1:33:18<16:27, 493.85s/it]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 105722 rows, 133 columns and 603468 nonzeros
Model fingerprint: 0xab9718ce
Variable types: 11 continuous, 122 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 1249901.0000
Presolve removed 90823 rows and 45 columns
Presolve time: 0.82s
Presolved: 14899 rows, 88 columns, 161090 nonzeros
Variable types: 10 continuous, 78 integer (78 binary)
Found heuristic solution: objective 669363.00000

Root relaxation: objective 2.017204e+05, 47 iterations, 0.08 seconds (0.11 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 201720.407    0   18 66936

 94%|█████████████████████████████████████▊  | 17/18 [1:38:07<07:12, 432.15s/it]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 211322 rows, 133 columns and 1206444 nonzeros
Model fingerprint: 0xd8482575
Variable types: 11 continuous, 122 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [9e+01, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 619242.00000
Presolve removed 210148 rows and 57 columns
Presolve time: 0.91s
Presolved: 1174 rows, 76 columns, 8098 nonzeros
Variable types: 10 continuous, 66 integer (66 binary)
Found heuristic solution: objective 311667.00000

Root relaxation: objective 2.334000e+03, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2334.00000    0   16 311667

100%|████████████████████████████████████████| 18/18 [1:38:12<00:00, 327.37s/it]


In [5]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln
json.dump(model_solutions, open('./primary_model_solutions.json', 'w'), indent=4)

In [6]:
s = 0
for model in primary_models:
    s += model.ObjVal
s

5855898.987111546

In [7]:
from itertools import permutations
K = [1,2,3]
print(*permutations(K,2))

(1, 2) (1, 3) (2, 1) (2, 3) (3, 1) (3, 2)
